In [9]:
from datetime import datetime

# Base Book class
class Book:
    def __init__(self, title, author, isbn, publication_year):
        self._title = title
        self._author = author
        self._isbn = isbn
        self._publication_year = publication_year
        self._is_borrowed = False
    
    @property
    def title(self):
        return self._title
    
    @property
    def author(self):
        return self._author
    
    @property
    def isbn(self):
        return self._isbn
    
    @property
    def publication_year(self):
        return self._publication_year
    
    @property
    def is_borrowed(self):
        return self._is_borrowed
    
    @is_borrowed.setter
    def is_borrowed(self, value):
        if isinstance(value, bool):
            self._is_borrowed = value
        else:
            raise ValueError("is_borrowed must be a boolean")
    
    def __str__(self):
        return f"'{self.title}' by {self.author} ({self.publication_year})"

# Specialized book classes using inheritance
class FictionBook(Book):
    def __init__(self, title, author, isbn, publication_year, genre):
        super().__init__(title, author, isbn, publication_year)
        self.genre = genre
        self.book_type = "Fiction"
    
    def __str__(self):
        return f"{super().__str__()} - {self.genre} {self.book_type}"

class NonFictionBook(Book):
    def __init__(self, title, author, isbn, publication_year, subject):
        super().__init__(title, author, isbn, publication_year)
        self.subject = subject
        self.book_type = "Non-Fiction"
    
    def __str__(self):
        return f"{super().__str__()} - {self.subject} {self.book_type}"

# Member class
class Member:
    def __init__(self, member_id, name, email):
        self.member_id = member_id
        self.name = name
        self.email = email
        self.borrowed_books = []
    
    def __str__(self):
        return f"Member: {self.name} (ID: {self.member_id})"

# Custom exceptions
class LibraryError(Exception):
    """Base exception for library-related errors"""
    pass

class BookNotAvailableError(LibraryError):
    """Raised when a book is not available for borrowing"""
    pass

class MemberLimitExceededError(LibraryError):
    """Raised when a member tries to borrow too many books"""
    pass

# Transaction class for tracking
class Transaction:
    def __init__(self, book, member, transaction_type):
        self.book = book
        self.member = member
        self.transaction_type = transaction_type  # 'borrow' or 'return'
        self.timestamp = datetime.now()
    
    def __str__(self):
        return f"{self.transaction_type.capitalize()}: {self.book.title} by {self.member.name} at {self.timestamp}"

# Main Library class
class Library:
    MAX_BOOKS_PER_MEMBER = 3
    
    def __init__(self, name):
        self.name = name
        self._books = []
        self._members = []
        self.transactions = []
    
    def add_book(self, book):
        # Use the actual class name for isinstance check
        if isinstance(book, (Book, FictionBook, NonFictionBook)):
            self._books.append(book)
            print(f"✓ Added: {book}")
            return True
        else:
            raise ValueError("Can only add Book objects")
    
    def register_member(self, member):
        if isinstance(member, Member):
            self._members.append(member)
            print(f"✓ Registered: {member}")
            return True
        else:
            raise ValueError("Can only register Member objects")
    
    def borrow_book(self, isbn, member_id):
        book = self._find_book_by_isbn(isbn)
        member = self._find_member_by_id(member_id)
        
        if not book:
            raise ValueError(f"Book with ISBN {isbn} not found")
        if not member:
            raise ValueError(f"Member with ID {member_id} not found")
        
        if book.is_borrowed:
            raise BookNotAvailableError(f"'{book.title}' is already borrowed")
        
        if len(member.borrowed_books) >= self.MAX_BOOKS_PER_MEMBER:
            raise MemberLimitExceededError(f"{member.name} has reached the borrowing limit of {self.MAX_BOOKS_PER_MEMBER} books")
        
        book.is_borrowed = True
        member.borrowed_books.append(book)
        transaction = Transaction(book, member, "borrow")
        self.transactions.append(transaction)
        
        print(f"✓ {member.name} borrowed '{book.title}'")
        return transaction
    
    def return_book(self, isbn, member_id):
        book = self._find_book_by_isbn(isbn)
        member = self._find_member_by_id(member_id)
        
        if not book:
            raise ValueError(f"Book with ISBN {isbn} not found")
        if not member:
            raise ValueError(f"Member with ID {member_id} not found")
        
        if book not in member.borrowed_books:
            raise ValueError(f"{member.name} didn't borrow '{book.title}'")
        
        book.is_borrowed = False
        member.borrowed_books.remove(book)
        transaction = Transaction(book, member, "return")
        self.transactions.append(transaction)
        
        print(f"✓ {member.name} returned '{book.title}'")
        return transaction
    
    def _find_book_by_isbn(self, isbn):
        for book in self._books:
            if book.isbn == isbn:
                return book
        return None
    
    def _find_member_by_id(self, member_id):
        for member in self._members:
            if member.member_id == member_id:
                return member
        return None
    
    def get_available_books(self):
        return [book for book in self._books if not book.is_borrowed]
    
    def get_borrowed_books(self):
        return [book for book in self._books if book.is_borrowed]
    
    def display_books(self):
        print(f"\n📚 --- Books in {self.name} ---")
        for i, book in enumerate(self._books, 1):
            status = "✅ Available" if not book.is_borrowed else "❌ Borrowed"
            print(f"{i}. {book} - {status}")
    
    def display_members(self):
        print(f"\n👥 --- Members of {self.name} ---")
        for i, member in enumerate(self._members, 1):
            print(f"{i}. {member}")
            if member.borrowed_books:
                print(f"   Borrowed books: {[book.title for book in member.borrowed_books]}")
    
    def display_transactions(self):
        print(f"\n📋 --- Recent Transactions in {self.name} ---")
        for i, transaction in enumerate(self.transactions[-5:], 1):  # Show last 5 transactions
            print(f"{i}. {transaction}")



In [10]:
# Test the complete system
def test_library_system():
    print("🚀 Testing Library Management System\n")
    
    # Create library
    library = Library("City Central Library")
    
    # Create and add books
    book1 = FictionBook("1984", "George Orwell", "1234567890", 1949, "Dystopian")
    book2 = NonFictionBook("Sapiens", "Yuval Noah Harari", "0987654321", 2011, "History")
    book3 = FictionBook("The Great Gatsby", "F. Scott Fitzgerald", "1122334455", 1925, "Classic")
    
    library.add_book(book1)
    library.add_book(book2)
    library.add_book(book3)
    
    # Register members
    member1 = Member("M001", "Alice Johnson", "alice@email.com")
    member2 = Member("M002", "Bob Smith", "bob@email.com")
    
    library.register_member(member1)
    library.register_member(member2)
    
    # Display initial state
    library.display_books()
    library.display_members()
    
    # Test borrowing
    print("\n" + "="*50)
    print("Testing Book Borrowing:")
    print("="*50)
    
    try:
        # Successful borrow
        library.borrow_book("1234567890", "M001")  # Alice borrows 1984
        library.borrow_book("0987654321", "M002")  # Bob borrows Sapiens
        
        # Try to borrow already borrowed book
        library.borrow_book("1234567890", "M002")  # Bob tries to borrow 1984
        
    except (BookNotAvailableError, MemberLimitExceededError, ValueError) as e:
        print(f"❌ Expected error: {e}")
    
    # Display current status
    library.display_books()
    library.display_members()
    
    # Test returning
    print("\n" + "="*50)
    print("Testing Book Returning:")
    print("="*50)
    
    try:
        # Return a book
        library.return_book("1234567890", "M001")  # Alice returns 1984
        
        # Now Bob can borrow it
        library.borrow_book("1234567890", "M002")  # Bob borrows 1984
        
    except (BookNotAvailableError, MemberLimitExceededError, ValueError) as e:
        print(f"❌ Error: {e}")
    
    # Test borrowing limit
    print("\n" + "="*50)
    print("Testing Borrowing Limit:")
    print("="*50)
    
    try:
        # Bob tries to borrow more than limit
        library.borrow_book("1122334455", "M002")  # Bob tries to borrow The Great Gatsby
        library.borrow_book("0987654321", "M002")  # Bob tries to borrow another book (should fail)
        
    except MemberLimitExceededError as e:
        print(f"❌ Expected limit error: {e}")
    
    # Final display
    library.display_books()
    library.display_members()
    library.display_transactions()
    
    # Show available books
    print(f"\n📖 Available books: {len(library.get_available_books())}")
    print(f"📚 Borrowed books: {len(library.get_borrowed_books())}")

# Run the test
if __name__ == "__main__":
    test_library_system()

🚀 Testing Library Management System

✓ Added: '1984' by George Orwell (1949) - Dystopian Fiction
✓ Added: 'Sapiens' by Yuval Noah Harari (2011) - History Non-Fiction
✓ Added: 'The Great Gatsby' by F. Scott Fitzgerald (1925) - Classic Fiction
✓ Registered: Member: Alice Johnson (ID: M001)
✓ Registered: Member: Bob Smith (ID: M002)

📚 --- Books in City Central Library ---
1. '1984' by George Orwell (1949) - Dystopian Fiction - ✅ Available
2. 'Sapiens' by Yuval Noah Harari (2011) - History Non-Fiction - ✅ Available
3. 'The Great Gatsby' by F. Scott Fitzgerald (1925) - Classic Fiction - ✅ Available

👥 --- Members of City Central Library ---
1. Member: Alice Johnson (ID: M001)
2. Member: Bob Smith (ID: M002)

Testing Book Borrowing:
✓ Alice Johnson borrowed '1984'
✓ Bob Smith borrowed 'Sapiens'
❌ Expected error: '1984' is already borrowed

📚 --- Books in City Central Library ---
1. '1984' by George Orwell (1949) - Dystopian Fiction - ❌ Borrowed
2. 'Sapiens' by Yuval Noah Harari (2011) - His

BookNotAvailableError: 'Sapiens' is already borrowed